In [1]:
import sagemaker

### 1. SageMaker Training/Hyperparameter Tuning Job Creation

In [5]:
from sagemaker.tensorflow import TensorFlow

In [25]:
sagemaker_role = sagemaker.get_execution_role()

# Creates the SageMaker training job using the TensorFlow estimator
sagemaker_estimator = TensorFlow(
    entry_point='train.py',
    source_dir='code',
    role=sagemaker_role,
    framework_version='2.2.0',
    model_dir='/opt/ml/model',
    py_version='py37',
    instance_type='ml.m5.4xlarge',
    instance_count=1,
    hyperparameters={
        'batch-size':16,
        'epochs':15
    },
#     output_path=s3_location
)

In [26]:
from sagemaker.tuner import ContinuousParameter, HyperparameterTuner

In [27]:
# Creates the SageMaker hyperparameter tuner
sagemaker_tuner = HyperparameterTuner(
    estimator=sagemaker_estimator,
    objective_metric_name='average test loss',
    hyperparameter_ranges={
        'learning-rate': ContinuousParameter(1e-4, 1e-3)
    },
    metric_definitions=[
        {
            'Name': 'average test loss',
            'Regex': 'Test Loss: ([0-9\\.]+)' # Corresponding line in code/train.py (168)
        }
    ],
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type='Minimize'
)

sagemaker_tuner.fit(inputs=channels)

.............................................!


### 3. SageMaker Endpoint Deployment

In [28]:
# Deploys a SageMaker hyperparameter endpoint
sagemaker_endpoint = sagemaker_tuner.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='imdb_sagemaker_endpoint'
)


2021-01-29 20:23:42 Starting - Preparing the instances for training
2021-01-29 20:23:42 Downloading - Downloading input data
2021-01-29 20:23:42 Training - Training image download completed. Training in progress.
2021-01-29 20:23:42 Uploading - Uploading generated training model
2021-01-29 20:23:42 Completed - Training job completed

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.



-----------------!

### 4. Resource Cleanup

In [30]:
sagemaker_endpoint.delete_endpoint()